In [ ]:
%pip install albumentations
%pip install roboflow

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="DLZRWqPmPgXpy0mevy0T")
project = rf.workspace("hundairotemtraining").project("image-detection-dxhgd")
version = project.version(1)
dataset = version.download("yolov8")
                

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.2/85.2 KB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 KB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 26.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 20.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 KB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 KB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.7/128.7 KB 32.6 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.11.0.86
    Uninstalling opencv-python-headless-4.11.0.86:
      Successfully uninstalled opencv-python-headless-4.11.0.86
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Image-Detection-1 in yolov8:: 100%|██████████| 28/28 [00:00<00:00, 13024.34it/s]


기본 테스트 코드

In [1]:
import albumentations as A
import cv2
import os


In [2]:
# 증강 파이프라인 설정
transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.3),
    A.Rotate(limit=10, p=0.3),
    A.GaussNoise(var_limit=(10.0, 50.0), p=0.2),
    A.MotionBlur(blur_limit=5, p=0.2),
    A.Resize(height=416, width=416),  # 안정적인 고정 리사이즈
], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))


/tmp/ipykernel_49135/4220190684.py:6: UserWarning: Argument(s) 'var_limit' are not valid for transform GaussNoise
  A.GaussNoise(var_limit=(10.0, 50.0), p=0.2),


In [4]:
# 경로 설정
input_img_dir = "./Image-Detection-1/train/images"
input_lbl_dir = "./Image-Detection-1/train/labels"

# 증강된 이미지와 레이블을 저장할 디렉토리
output_img_dir = "./Image-Detection-1/augmented_dataset/images"
output_lbl_dir = "./Image-Detection-1/augmented_dataset/labels"

os.makedirs(output_img_dir, exist_ok=True)
os.makedirs(output_lbl_dir, exist_ok=True)


In [6]:
for img_file in os.listdir(input_img_dir):
    if not img_file.endswith(('.jpg', '.png')):
        continue

    # 경로 준비
    img_path = os.path.join(input_img_dir, img_file)
    lbl_path = os.path.join(input_lbl_dir, img_file.rsplit('.', 1)[0] + '.txt')

    # 이미지 로드
    image = cv2.imread(img_path)
    h, w = image.shape[:2]

    # 라벨 로드
    bboxes = []
    class_labels = []
    if os.path.exists(lbl_path):
        with open(lbl_path, 'r') as f:
            for line in f:
                cls, x, y, bw, bh = line.strip().split()
                bboxes.append([float(x), float(y), float(bw), float(bh)])
                class_labels.append(cls)

    # 증강 적용
    try:
        augmented = transform(image=image, bboxes=bboxes, class_labels=class_labels)
        aug_img = augmented['image']
        aug_bboxes = augmented['bboxes']
        aug_labels = augmented['class_labels']
    except Exception as e:
        print(f"⚠️ {img_file} 증강 실패: {e}")
        continue

    # 저장 이름 만들기
    base_name = img_file.rsplit('.', 1)[0]
    aug_img_name = f"{base_name}_aug.jpg"
    aug_lbl_name = f"{base_name}_aug.txt"

    # 이미지 저장
    cv2.imwrite(os.path.join(output_img_dir, aug_img_name), aug_img)

    # 라벨 저장
    with open(os.path.join(output_lbl_dir, aug_lbl_name), 'w') as f:
        for bbox, label in zip(aug_bboxes, aug_labels):
            f.write(f"{label} {bbox[0]:.6f} {bbox[1]:.6f} {bbox[2]:.6f} {bbox[3]:.6f}\n")


## 날씨, 배경을 바꾸는 데이터 증강

In [19]:
import os
import cv2
import albumentations as A
from albumentations.augmentations import functional as F
import random

# 경로 설정
input_img_dir = "./Image-Detection-1/train/images"
input_lbl_dir = "./Image-Detection-1/train/labels"

# 증강된 이미지와 레이블을 저장할 디렉토리
output_img_dir = "./Image-Detection-1/augmented_dataset/images"
output_lbl_dir = "./Image-Detection-1/augmented_dataset/labels"

os.makedirs(output_img_dir, exist_ok=True)
os.makedirs(output_lbl_dir, exist_ok=True)


In [15]:
def add_rain(img):
    rain_layer = np.zeros_like(img, dtype=np.uint8)
    for _ in range(1500):
        x, y = np.random.randint(0, img.shape[1]), np.random.randint(0, img.shape[0])
        length = np.random.randint(10, 20)
        cv2.line(rain_layer, (x, y), (x - 3, y + length), (200, 200, 200), 1)
    blended = cv2.addWeighted(img, 0.8, rain_layer, 0.2, 0)
    return blended

def add_snow(img):
    snow_layer = np.zeros_like(img, dtype=np.uint8)
    for _ in range(2000):
        x, y = np.random.randint(0, img.shape[1]), np.random.randint(0, img.shape[0])
        cv2.circle(snow_layer, (x, y), radius=1, color=(255, 255, 255), thickness=-1)
    blended = cv2.addWeighted(img, 0.85, snow_layer, 0.25, 0)
    return blended

def add_wind(img):
    blurred = cv2.GaussianBlur(img, (11, 3), sigmaX=10)
    noise = np.random.normal(0, 5, img.shape).astype(np.uint8)
    windy = cv2.add(blurred, noise)
    return windy


In [24]:
#⚡ 1. 번개 치는 날 (Thunderstorm)
# 야간 + 밝은 섬광 효과 (랜덤한 강한 밝기 영역)
def add_thunderstorm(img):
    img = cv2.convertScaleAbs(img, alpha=0.5, beta=-30)  # 어둡게
    mask = np.zeros_like(img, dtype=np.uint8)
    for _ in range(2):
        x, y = np.random.randint(0, img.shape[1]), np.random.randint(0, img.shape[0])
        radius = np.random.randint(50, 120)
        cv2.circle(mask, (x, y), radius, (255, 255, 255), -1)
    lightning = cv2.addWeighted(img, 1.0, mask, 0.6, 0)
    return lightning

In [25]:
# 🌫️ 2. 안개 낀 날 (Foggy)
# 채도를 줄이고 흐릿한 느낌 (흐린 알파블렌딩)
def add_fog(img):
    fog = cv2.GaussianBlur(img, (51, 51), 0)
    foggy = cv2.addWeighted(img, 0.6, fog, 0.4, 0)
    return foggy

In [26]:
# 🌈 3. 무지개 뜬 날 (Rainbow)
# 화려한 색상의 아크를 투명하게 합성
def add_rainbow(img):
    rainbow = np.zeros_like(img)
    h, w = img.shape[:2]
    center = (w // 2, h)
    for i, color in enumerate([(255, 0, 0), (255, 127, 0), (255, 255, 0), 
                               (0, 255, 0), (0, 0, 255), (75, 0, 130), (143, 0, 255)]):
        cv2.ellipse(rainbow, center, (int(w*0.8)-i*8, int(h*0.5)-i*8), 0, 0, 180, color, thickness=6)
    rainbowed = cv2.addWeighted(img, 1.0, rainbow, 0.3, 0)
    return rainbowed

In [27]:
# 🌙 4. 밤하늘 (Night time)
# 채도와 밝기를 줄이고 파란 빛
def add_night(img):
    dark = cv2.convertScaleAbs(img, alpha=0.5, beta=-50)
    night = cv2.addWeighted(dark, 1.0, np.full_like(img, (10, 10, 30)), 0.3, 0)
    return night

In [28]:
# 5.🌪️ 회오리바람 효과 함수
# 회오리 바람은 시각적으로 강한 방사형 왜곡, 살짝의 흐림, 그리고 약간의 회전된 노이즈/먼지 효과를 조합

def add_tornado(img):
    h, w = img.shape[:2]
    
    # 회전 왜곡을 위한 좌표 그리드 생성
    center = (w // 2, h // 2)
    strength = 0.0008  # 회오리 강도 (조절 가능)

    # 좌표 맵 생성
    map_x = np.zeros((h, w), dtype=np.float32)
    map_y = np.zeros((h, w), dtype=np.float32)

    for y in range(h):
        for x in range(w):
            dx = x - center[0]
            dy = y - center[1]
            r = np.sqrt(dx**2 + dy**2)
            theta = np.arctan2(dy, dx) + strength * r
            map_x[y, x] = center[0] + r * np.cos(theta)
            map_y[y, x] = center[1] + r * np.sin(theta)

    # 회전 왜곡 적용
    swirl = cv2.remap(img, map_x, map_y, interpolation=cv2.INTER_LINEAR, borderMode=cv2.BORDER_REFLECT)

    # 블러와 노이즈로 현실감 추가
    swirl_blur = cv2.GaussianBlur(swirl, (5, 5), sigmaX=3)
    noise = np.random.normal(0, 15, img.shape).astype(np.uint8)
    tornado = cv2.add(swirl_blur, noise)

    return tornado


In [ ]:
# 6.🍂 낙엽 날리는 효과 함수 (add_falling_leaves)
def add_falling_leaves(img, num_leaves=30):
    h, w = img.shape[:2]
    leaf_img = np.zeros_like(img)

    for _ in range(num_leaves):
        # 낙엽 색상 랜덤
        color = random.choice([(165, 42, 42), (255, 140, 0), (255, 215, 0), (139, 69, 19)])  # 갈색, 주황, 노랑, 짙은 갈색
        
        # 크기, 위치, 회전 각도 랜덤
        center = (random.randint(0, w), random.randint(0, h))
        size = random.randint(10, 30)
        angle = random.uniform(0, 360)
        
        # 타원 그리기 (낙엽 형태)
        # 색상이 랜덤임
        axes = (size, int(size * 0.4))
        temp = np.zeros_like(img)
        cv2.ellipse(temp, center, axes, angle, 0, 360, color, -1)

        # 누적
        alpha = random.uniform(0.3, 0.7)
        leaf_img = cv2.addWeighted(leaf_img, 1.0, temp, alpha, 0)

    # 낙엽 합성
    result = cv2.addWeighted(img, 1.0, leaf_img, 0.4, 0)
    return result

In [36]:
def get_weather_transform(weather):
    transforms = [A.Resize(640, 640)]
    post_effect = None

    if weather == 'sunny':
        transforms.append(
            A.RandomBrightnessContrast(brightness_limit=(0.2, 0.5), contrast_limit=0.2, p=1.0)
        )

    elif weather == 'dark':
        transforms.extend([
            A.RandomBrightnessContrast(brightness_limit=(-0.6, -0.3), contrast_limit=0.1, p=1.0),
            A.HueSaturationValue(hue_shift_limit=10, sat_shift_limit=-30, val_shift_limit=-30, p=1.0)
        ])

    elif weather == 'sandstorm':
        transforms.extend([
            A.RandomBrightnessContrast(brightness_limit=(0.0, 0.1), contrast_limit=0.1, p=1.0),
            A.HueSaturationValue(hue_shift_limit=20, sat_shift_limit=20, val_shift_limit=-10, p=1.0),
            A.MotionBlur(p=0.5)
        ])

    elif weather == 'waves':
        transforms.extend([
            A.Blur(blur_limit=3, p=0.3),
            A.GaussNoise(var_limit=(20.0, 50.0), p=0.5),
            A.HueSaturationValue(hue_shift_limit=10, sat_shift_limit=30, val_shift_limit=-20, p=1.0)
        ])

    elif weather in ['rainy', 'snowy', 'windy', 'thunderstorm', 'foggy', 'rainbow', 'night', 'tornado', 'falling_leaves']:
        post_effect = weather  # 후처리용 플래그 설정

    return A.Compose(transforms, bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels'])), post_effect


In [37]:
weather_types = [
    'sunny', 'dark', 'sandstorm', 'waves',
    'rainy', 'snowy', 'windy',
    'thunderstorm', 'foggy', 'rainbow', 'night',
    'tornado', 'falling_leaves'
]

In [38]:
for img_file in os.listdir(input_img_dir):
    if not img_file.endswith(('.jpg', '.png')):
        continue

    img_path = os.path.join(input_img_dir, img_file)
    lbl_path = os.path.join(input_lbl_dir, img_file.rsplit('.', 1)[0] + '.txt')

    image = cv2.imread(img_path)
    h, w = image.shape[:2]

    # Resize for YOLO
    image = cv2.resize(image, (640, 640))

    bboxes = []
    class_labels = []
    if os.path.exists(lbl_path):
        with open(lbl_path, 'r') as f:
            for line in f:
                cls, x, y, bw, bh = line.strip().split()
                bboxes.append([float(x), float(y), float(bw), float(bh)])
                class_labels.append(cls)

    for weather in weather_types:
        transform = get_weather_transform(weather)

        transform, post_effect = get_weather_transform(weather)

        try:
            augmented = transform(image=image, bboxes=bboxes, class_labels=class_labels)
            aug_img = augmented['image']
            aug_bboxes = augmented['bboxes']
            aug_labels = augmented['class_labels']
        except Exception as e:
            print(f"⚠️ {img_file} {weather} 증강 실패: {e}")
            continue

        # 후처리 함수 호출
        if post_effect == 'rainy':
            aug_img = add_rain(aug_img)
        elif post_effect == 'snowy':
            aug_img = add_snow(aug_img)
        elif post_effect == 'windy':
            aug_img = add_wind(aug_img)
        elif post_effect == 'thunderstorm':
            aug_img = add_thunderstorm(aug_img)
        elif post_effect == 'foggy':
            aug_img = add_fog(aug_img)
        elif post_effect == 'rainbow':
            aug_img = add_rainbow(aug_img)
        elif post_effect == 'night':
            aug_img = add_night(aug_img)
        elif post_effect == 'tornado':
            aug_img = add_tornado(aug_img)
        elif post_effect == 'falling_leaves':
            aug_img = add_falling_leaves(aug_img)

        # 저장
        base_name = img_file.rsplit('.', 1)[0]
        aug_img_name = f"{base_name}_{weather}.jpg"
        aug_lbl_name = f"{base_name}_{weather}.txt"

        cv2.imwrite(os.path.join(output_img_dir, aug_img_name), aug_img)

        with open(os.path.join(output_lbl_dir, aug_lbl_name), 'w') as f:
            for bbox, label in zip(aug_bboxes, aug_labels):
                f.write(f"{label} {bbox[0]:.6f} {bbox[1]:.6f} {bbox[2]:.6f} {bbox[3]:.6f}\n")

/tmp/ipykernel_12340/211686332.py:26: UserWarning: Argument(s) 'var_limit' are not valid for transform GaussNoise
  A.GaussNoise(var_limit=(20.0, 50.0), p=0.5),
